In [176]:
import os, random
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfTransformer
import pickle
import matplotlib as plt
from nilearn import plotting, datasets
random.seed(42)

## 1. Train models to predict brain activations from text-based psychological features

### Functions for training and prediction

In [2]:
def train(classifier, activation_atlas, features, verbose=False):
    fits, count = {}, 0
    for structure in activation_atlas.columns:
        if verbose:
            clear_output()
            print("Fitting classifier {}/{}: {}".format(count+1, len(activation_atlas.columns), structure))
        classifier.fit(features, activation_atlas[structure])
        fits[structure] = pickle.dumps(classifier)
    if verbose:
        print("Trained classifiers for {} brain structures".format(count))
    return fits

In [3]:
def mind2brain(fits, activation_atlas, features):
    predictions = {}
    for structure in activation_atlas.columns:
        fit = pickle.loads(fits[structure])
        predictions[structure] = fit.predict(features)
    return predictions

### Training and test sets of activation and text data

In [188]:
activation_atlas = pd.read_csv("data/activations_prob0.csv", index_col=0, header=0)
features = pd.read_csv("data/tokens_ontol.csv.gz", index_col=0, header=0)

In [86]:
train_prop, val_prop = 0.7, 0.2
ids = sorted(list(activation_atlas.index))
random.shuffle(ids)
train_ids = ids[:int(train_prop*len(ids))]
test_ids = ids[int((train_prop+val_prop)*len(ids)):]

### Fit the models with the optimal parameters

In [9]:
classifier = LogisticRegression(penalty="l2",
                                C=0.090989672, 
                                fit_intercept=True,
                                max_iter=200)

In [10]:
fits = train(classifier, activation_atlas.loc[train_ids], features.loc[train_ids], verbose=True)

Trained 56 classifiers with 0 failure(s)


## 2. Evaluate the models on neuroimaging articles withheld from training

### Overall test set performance

In [11]:
pred = pd.DataFrame(mind2brain(fits, activation_atlas.loc[test_ids], features.loc[test_ids], verbose=True))
print("\nOverall performance on the test set\n")
print("PRECISION:\t {0:.3f}".format(precision_score(activation_atlas.loc[test_ids], pred, average="micro")))
print("RECALL:\t\t {0:.3f}".format(recall_score(activation_atlas.loc[test_ids], pred, average="micro")))
print("F1:\t\t {0:.3f}".format(f1_score(activation_atlas.loc[test_ids], pred, average="micro")))


Overall performance on the test set

PRECISION:	 0.610
RECALL:		 0.483
F1:		 0.539


### Performance on a randomly selected article

In [12]:
PMID = random.sample(test_ids, 1)

In [13]:
pd.set_option("display.max_columns", len(features.columns))
features.loc[PMID]

,2ndorder_rule_acquisition,2stage_decision_task,abductive_reasoning,abductive_reasoning_activity,abductive_reasoning_function,aberrant_behavior_checklist_community,abstract_analogy,abstract_knowledge,abstractconcrete_judgment_bilingual,abstractconcrete_task,accelleration_stimulus_transduction,acoustic_coding,acoustic_encoding,acoustic_phonetic_processing,acoustic_processing,acquired_equivalence,action,action_imitation_task,action_initiation,action_observation_task,action_perception,actionperception_loop,activation,activation_level,active_maintenance,active_recall,active_retrieval,acuity,acupuncture,acupuncture_task,adaptation,adaptation_of_marshmellow_test,adaptive_control,adaptive_nback_task,addiction,adolescent_symptom_inventory,adult_adhd_clinical_diagnostic_scale,adult_adhd_selfreport_scale,adult_attachment_interview,adult_behavior_checklist,adult_penn_word_memory_test_delayed_memory,affect_perception,affect_recognition,affective_picture,affective_word,agency,agreeableness,air_hunger,alexithymia,alternating_run_paradigm,altruism,altruistic_motivation,alveolar,ambiguous_figure_task,american_national_adult_reading_test,amodal_representation,analog_representation,analogical_encoding,analogical_inference,analogical_problem_solving,analogical_reasoning,analogical_reasoning_task,analogical_transfer,analogy,anchoring,anger,angling_risk_task,anhedonia,animacy_decision,animacy_perception,animal_naming_task,ant_task,anticipation,antisaccadeprosaccade_task,antisaccades,antisocial_personality,anxiety,apparent_motion,apparent_verticality_judgment,apperception,appetite,appetitive_function,appetitive_motivation,arithmetic_processing,arousal,arousal_emotion,arousal_physical,articulation,articulatory_loop,articulatory_planning,articulatory_rehearsal,articulatory_suppression_task,artificial_grammar_learning_task,assimilation,association,association_learning,associative_memory_encoding_task,associative_priming,ataxia,attachment,attended_channel,attended_stimulus,attention,attention_bias,attention_capacity,attention_network_test,attention_shift,attention_shifting,attention_span,attention_switching_task,attentional_bias,attentional_blink,attentional_blink_paradigm,attentional_effort,attentional_focusing,attentional_resource,attentional_state,attitude,audio_narrative,audiovisual_perception,audiovisual_target_detection_task,audition,auditory_arithmetic_processing,auditory_attention,auditory_coding,auditory_encoding,auditory_feedback,auditory_grouping,auditory_imagery,auditory_learning,auditory_lexical_access,auditory_localization,auditory_masking,auditory_masking_task,auditory_memory,auditory_perception,auditory_recognition,auditory_scene,auditory_scene_analysis,auditory_scene_perception,auditory_sensation,auditory_sentence_comprehension,auditory_sentence_recognition,auditory_stream_segregation,auditory_system_function,auditory_temporal_discrimination_task,auditory_tone_detection,auditory_tone_discrimination,auditory_tone_perception,auditory_word_comprehension,auditory_word_recognition,auditory_working_memory,autism_diagnostic_interview_revised,autism_diagnostic_observation_schedule,autism_spectrum_quotient,autobiographical_memory,autobiographical_memory_task,autobiographical_recall,automaticity,autonoesis,autonomic_nervous_system_function,availability_heuristic,aversive_learning,axcpt_task,axdpx,backward_chaining,backward_digit_span_task,backward_masking,balance,balloon_analogue_risk_task,baroregulation,barratt_impulsiveness_scale,battelle_developmental_inventory,beckerdegrootmarschak_bdm_procedure,beerybuktenica_developmental_test_of_visualmotor_integration,behavioral_activity,behavioral_approachinhibition_system,behavioral_inhibition,behavioral_inhibition_cognitive,behavioral_inhibition_temperament,behavioral_investment_allocation_strategy,behavioral_process,behavioral_rating_inventory_of_executive_function,behavioral_system_function,belief,benton_facial_recognition_test,bicarbonate_stimulus_transduction,bickel_titrator,big_five_questionnaire

In [14]:
pred = mind2brain(fits, activation_atlas.loc[PMID], features.loc[PMID], verbose=True)

In [15]:
pd.set_option("display.max_columns", len(activation_atlas.columns))
pred_vs_true = pd.DataFrame(pred).append(activation_atlas.loc[PMID])
pred_vs_true.index = ["PREDICTED", "TRUE"]
pred_vs_true

,accumbens,amygdala,angular_gyrus,brainstem,caudate,central_opercular_cortex,cingulate_gyrus_anterior_division,cingulate_gyrus_posterior_division,cuneal_cortex,frontal_medial_cortex,frontal_operculum_cortex,frontal_orbital_cortex,frontal_pole,heschls_gyrus,hippocampus,inferior_frontal_gyrus_pars_opercularis,inferior_frontal_gyrus_pars_triangularis,inferior_temporal_gyrus_anterior_division,inferior_temporal_gyrus_posterior_division,inferior_temporal_gyrus_temporooccipital_part,insular_cortex,intracalcarine_cortex,lateral_occipital_cortex_inferior_division,lateral_occipital_cortex_superior_division,lingual_gyrus,middle_frontal_gyrus,middle_temporal_gyrus_anterior_division,middle_temporal_gyrus_posterior_division,middle_temporal_gyrus_temporooccipital_part,occipital_fusiform_gyrus,occipital_pole,pallidum,paracingulate_gyrus,parahippocampal_gyrus_anterior_division,parahippocampal_gyrus_posterior_division,parietal_operculum_cortex,planum_polare,planum_temporale,postcentral_gyrus,precentral_gyrus,precuneous_cortex,putamen,subcallosal_cortex,superior_frontal_gyrus,superior_parietal_lobule,superior_temporal_gyrus_anterior_division,superior_temporal_gyrus_posterior_division,supplementary_motor_cortex,supracalcarine_cortex,supramarginal_gyrus_anterior_division,supramarginal_gyrus_posterior_division,temporal_fusiform_cortex_anterior_division,temporal_fusiform_cortex_posterior_division,temporal_occipital_fusiform_cortex,temporal_pole,thalamus
PREDICTED,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0
TRUE,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0


In [16]:
print("\nPerformance on a randomly selected article (PMID={})\n".format(PMID[0]))
print("PRECISION:\t {0:.3f}".format(precision_score(pred_vs_true.loc["TRUE"], pred_vs_true.loc["PREDICTED"])))
print("RECALL:\t\t {0:.3f}".format(recall_score(pred_vs_true.loc["TRUE"], pred_vs_true.loc["PREDICTED"])))
print("F1:\t\t {0:.3f}".format(f1_score(pred_vs_true.loc["TRUE"], pred_vs_true.loc["PREDICTED"])))


Performance on a randomly selected article (PMID=16751298)

PRECISION:	 0.500
RECALL:		 0.583
F1:		 0.538


## 3. Visualize relationships between psychological features and brain activity

### Reweight features by inverse document frequency

In [190]:
tfidf = TfidfTransformer(norm = None, use_idf = True, smooth_idf = True, sublinear_tf = False)

In [192]:
features_complete = features.dropna(axis=1, how="all")
feat_tfidf = tfidf.fit_transform(features_complete)
feat_tfidf = pd.DataFrame(feat_tfidf.toarray(), index=features_complete.index, columns=features_complete.columns)

### Cosine similarity across the corpus

In [200]:
cossim = cosine_similarity(activation_atlas.transpose(), feat_tfidf.transpose())


In [201]:
cossim

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])